In [ ]:
import re
import pandas as pd
import nltk
import string
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('words')
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords, words
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


In [ ]:
!unzip /content/Twitter_Sentiments.zip

Archive:  /content/Twitter_Sentiments.zip
  inflating: Twitter Sentiments.csv  


In [ ]:
df = pd.read_csv('/content/Twitter Sentiments.csv')
df.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [ ]:
def text_preprocessing(text):

  text = text.lower()

  text = re.sub(r'[^a-zA-Z]', ' ', text)

  text = re.sub(r'@[\w]*', '', text)

  tokens = word_tokenize(text)

  stop_words = set(stopwords.words('english'))
  tokens = [token for token in tokens if token not in stop_words]

  punctuations = string.punctuation
  tokens = [token for token in tokens if token not in punctuations]

  #actualwords = set(words.words())
  #actual_words = [token for token in tokens if token in actualwords]
  #it is taking huge time to run because of this O(n) time complexity

  #tokens = [TextBlob(token).correct() for token in tokens]
  #corrected_tokens = [str(token) for token in tokens]
  #I tried running the code with TextBlob but it is taking huge time to run because of O(n) time complexity for TextBlob operation i guess

  lemmatizer = WordNetLemmatizer()
  #words = [lemmatizer.lemmatize(token) for token in corrected_tokens]
  allwords = [lemmatizer.lemmatize(token) for token in tokens]

  preprocessed_text = ' '.join(allwords)

  return   preprocessed_text



In [ ]:
df['tweet'] = df['tweet'].apply(text_preprocessing)

In [ ]:
df.head()

,id,label,tweet
0,1,0,user father dysfunctional selfish drag kid dys...
1,2,0,user user thanks lyft credit use cause offer w...
2,3,0,bihday majesty
3,4,0,model love u take u time ur
4,5,0,factsguide society motivation


In [103]:
cv = CountVectorizer()
bow = cv.fit_transform(df.tweet)
column_names = cv.get_feature_names_out()
english_words = set(words.words())
relevant_column_names = [word for word in column_names if word in english_words]
new_bow = bow[:, [cv.vocabulary_.get(word) for word in relevant_column_names]]
bow_df = pd.DataFrame(new_bow.toarray(), columns= relevant_column_names)

In [104]:
bow_df.head()

,aa,aal,aba,abandon,abandoned,abbey,abdominal,aberrant,abiding,ability,...,zing,zip,zodiac,zombie,zone,zoo,zoological,zoologist,zorro,zucchini
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [106]:
final_df = pd.concat([df, bow_df], axis=1)
final_df.head()
#This is the final dataframe containing the original preprocessed csv dataset and the corresponding bow

,id,label,tweet,aa,aal,aba,abandon,abandoned,abbey,abdominal,...,zing,zip,zodiac,zombie,zone,zoo,zoological,zoologist,zorro,zucchini
0,1,0,user father dysfunctional selfish drag kid dys...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,user user thanks lyft credit use cause offer w...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,bihday majesty,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,model love u take u time ur,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,0,factsguide society motivation,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
